# K₇ Spectral Analysis v3: Analytical G₂ Metric

**Approach**: True Laplace-Beltrami operator on grid with analytical G₂ metric.

## What's Different from v1/v2

| v1/v2 (Broken) | v3 (This) |
|----------------|----------|
| Point cloud sampling | **Grid discretization** |
| Graph Laplacian | **Laplace-Beltrami operator** |
| Arbitrary k-NN | **Finite differences** |
| S³ calibration | **T⁷ flat calibration** |
| Unknown scaling | **Analytical metric from gift-core** |

## The Key Insight

From `gift-core/data/analytical_g2_metric.json`:
- **Metric is diagonal and CONSTANT**: g_ij = (65/32)^(1/7) × δ_ij
- **det(g) = 65/32** exactly (topological constraint)
- **Torsion = 0** (constant form)

This means K₇ with the analytical G₂ metric is **conformally flat**!

---

In [ ]:
# @title 1. Setup
%%capture
!pip install -q scipy numpy matplotlib

In [ ]:
# @title 2. Imports & Constants
import numpy as np
from scipy.sparse.linalg import LinearOperator, eigsh
from scipy import sparse
import time
import json
import warnings
warnings.filterwarnings('ignore')

# GIFT Constants (from gift-core)
DIM_G2 = 14
DIM_K7 = 7
B2 = 21
B3 = 77
H_STAR = B2 + B3 + 1  # = 99

# Analytical G₂ metric from gift-core/data/analytical_g2_metric.json
DET_G_TARGET = 65 / 32  # = 2.03125 exactly
SCALE_FACTOR_METRIC = (65/32) ** (1/14)  # ≈ 1.0543
G_DIAGONAL = (65/32) ** (1/7)  # ≈ 1.1115 (each diagonal element)

print("═" * 60)
print("GIFT ANALYTICAL G₂ METRIC")
print("═" * 60)
print(f"det(g) target = 65/32 = {DET_G_TARGET}")
print(f"g_ii = (65/32)^(1/7) = {G_DIAGONAL:.6f}")
print(f"Verification: g_ii^7 = {G_DIAGONAL**7:.6f} ≈ {DET_G_TARGET}")
print(f"\nH* = {H_STAR}")
print(f"dim(G₂) = {DIM_G2}")

In [ ]:
# @title 3. Metric Functions

def metric_flat(x: np.ndarray) -> np.ndarray:
    """
    Flat metric: g_ij = δ_ij
    Used for T⁷ calibration.
    """
    if x.ndim == 1:
        return np.ones(7)
    return np.ones((len(x), 7))

def metric_g2_analytical(x: np.ndarray) -> np.ndarray:
    """
    Analytical G₂ metric from GIFT:
    g_ij = (65/32)^(1/7) × δ_ij
    
    This is CONSTANT (independent of x).
    det(g) = (65/32)^(7/7) = 65/32 exactly.
    """
    if x.ndim == 1:
        return np.full(7, G_DIAGONAL)
    return np.full((len(x), 7), G_DIAGONAL)

def metric_g2_modulated(x: np.ndarray, amplitude: float = 0.1) -> np.ndarray:
    """
    G₂ metric with smooth position-dependent modulation.
    Preserves det(g) = 65/32 on average.
    """
    if x.ndim == 1:
        x = x.reshape(1, -1)
    
    # Base metric
    g = np.full((len(x), 7), G_DIAGONAL)
    
    # Add smooth modulation (preserves average det)
    for d in range(7):
        # Modulate each direction with different phases
        phase = 2 * np.pi * d / 7
        modulation = 1 + amplitude * np.sin(2 * np.pi * x[:, d] + phase)
        g[:, d] *= modulation
    
    # Normalize to preserve det(g) = 65/32
    current_det = np.prod(g, axis=1, keepdims=True)
    target_det = DET_G_TARGET
    g *= (target_det / current_det) ** (1/7)
    
    return g if len(g) > 1 else g[0]

print("Metric functions defined.")
print(f"  metric_flat: g_ii = 1, det(g) = 1")
print(f"  metric_g2_analytical: g_ii = {G_DIAGONAL:.4f}, det(g) = {DET_G_TARGET}")
print(f"  metric_g2_modulated: position-dependent, avg det(g) = {DET_G_TARGET}")

In [ ]:
# @title 4. Matrix-Free Curved Laplacian

def apply_laplacian_7d(
    v: np.ndarray,
    grid_shape: tuple,
    metric_func,
    h: float = 1.0
) -> np.ndarray:
    """
    Apply curved Laplacian using finite differences (periodic BC).
    
    Δ_g f = (1/√det(g)) ∂_i(√det(g) g^{ii} ∂_i f)
    
    For diagonal metric with constant g_ii, this simplifies!
    """
    dim = len(grid_shape)
    u = v.reshape(grid_shape)
    
    # For constant diagonal metric, the Laplacian simplifies to:
    # Δ_g f = (1/g_ii) ∂_i² f  (sum over i, no cross terms)
    # Since our metric is constant, we can precompute g_ii
    
    # Get metric at origin (it's constant anyway)
    g_diag = metric_func(np.zeros(dim))
    
    Lv = np.zeros_like(u)
    h2 = h * h
    
    for d in range(dim):
        # Second derivative: (u[i+1] - 2u[i] + u[i-1]) / h²
        u_plus = np.roll(u, -1, axis=d)
        u_minus = np.roll(u, 1, axis=d)
        
        # Laplacian contribution from dimension d
        # Δ_d = (1/g_dd) × d²/dx_d²
        Lv += (u_plus - 2*u + u_minus) / (h2 * g_diag[d])
    
    return Lv.ravel()

def create_laplacian_operator(
    grid_shape: tuple,
    metric_func,
    domain_size: float = 2*np.pi
) -> LinearOperator:
    """
    Create LinearOperator for scipy.sparse.linalg.eigsh.
    
    Returns -Δ_g (positive semi-definite for eigenvalue computation).
    """
    N = np.prod(grid_shape)
    h = domain_size / grid_shape[0]  # Uniform grid spacing
    
    def matvec(v):
        # Return -Δ_g v (positive eigenvalues)
        return -apply_laplacian_7d(v, grid_shape, metric_func, h)
    
    return LinearOperator(
        shape=(N, N),
        matvec=matvec,
        rmatvec=matvec,  # Symmetric
        dtype=np.float64
    )

print("Matrix-free Laplacian operator ready.")

In [ ]:
# @title 5. Eigenvalue Computation

def compute_spectrum(
    grid_size: int,
    metric_func,
    n_eigenvalues: int = 10,
    domain_size: float = 2*np.pi
) -> dict:
    """
    Compute smallest eigenvalues of -Δ_g on 7D torus.
    """
    grid_shape = (grid_size,) * 7
    N_total = grid_size ** 7
    
    print(f"Grid: {grid_size}^7 = {N_total:,} points")
    
    # Create operator
    t0 = time.time()
    L = create_laplacian_operator(grid_shape, metric_func, domain_size)
    
    # Compute eigenvalues
    # 'SM' = smallest magnitude, but our -Δ is positive semi-definite
    # so smallest magnitude = smallest eigenvalues
    n_compute = min(n_eigenvalues + 5, N_total - 1)
    eigenvalues, _ = eigsh(L, k=n_compute, which='SM', tol=1e-6, maxiter=5000)
    eigenvalues = np.sort(eigenvalues)
    
    elapsed = time.time() - t0
    
    # First nonzero eigenvalue
    nonzero = eigenvalues[eigenvalues > 1e-8]
    lambda1 = nonzero[0] if len(nonzero) > 0 else eigenvalues[1]
    
    print(f"  Time: {elapsed:.2f}s")
    print(f"  λ₀ = {eigenvalues[0]:.6f} (should be ~0)")
    print(f"  λ₁ = {lambda1:.6f}")
    
    return {
        'grid_size': grid_size,
        'eigenvalues': eigenvalues[:n_eigenvalues],
        'lambda1': lambda1,
        'time': elapsed
    }

print("Spectrum computation ready.")

In [ ]:
# @title 6. Calibration on T⁷ Flat

print("═" * 60)
print("CALIBRATION ON T⁷ (FLAT TORUS)")
print("═" * 60)
print("\nTheory: On T⁷ with period 2π and flat metric,")
print("  λ_n = Σᵢ nᵢ² where nᵢ ∈ ℤ")
print("  λ₁ = 1 (e.g., n = (1,0,0,0,0,0,0))")

# Test on small grid first
print("\n--- Grid size = 5 ---")
result_flat_5 = compute_spectrum(5, metric_flat, n_eigenvalues=10)

print("\n--- Grid size = 7 ---")
result_flat_7 = compute_spectrum(7, metric_flat, n_eigenvalues=10)

# Calibration factor
# Theory: λ₁(T⁷, flat) = 1
# Numerical: λ₁_numerical
# Scale factor: true / numerical
lambda1_flat_theory = 1.0
calibration_factor_5 = lambda1_flat_theory / result_flat_5['lambda1']
calibration_factor_7 = lambda1_flat_theory / result_flat_7['lambda1']

print(f"\n--- Calibration Factors ---")
print(f"  Grid 5: λ₁_num = {result_flat_5['lambda1']:.6f}, factor = {calibration_factor_5:.6f}")
print(f"  Grid 7: λ₁_num = {result_flat_7['lambda1']:.6f}, factor = {calibration_factor_7:.6f}")
print(f"  Ratio: {calibration_factor_5/calibration_factor_7:.4f} (should → 1 as grid refines)")

In [ ]:
# @title 7. Main Experiment: K₇ with Analytical G₂ Metric

print("\n" + "═" * 60)
print("K₇ WITH ANALYTICAL G₂ METRIC")
print("═" * 60)
print(f"\nMetric: g_ii = (65/32)^(1/7) = {G_DIAGONAL:.6f}")
print(f"det(g) = 65/32 = {DET_G_TARGET}")

# For constant diagonal metric g_ii = c:
# Δ_g = (1/c) Δ_flat
# So eigenvalues scale: λ_g = λ_flat / c
print(f"\nTheory for constant diagonal metric:")
print(f"  λ₁(g) = λ₁(flat) / g_ii = 1 / {G_DIAGONAL:.6f} = {1/G_DIAGONAL:.6f}")

print("\n--- Grid size = 5 ---")
result_g2_5 = compute_spectrum(5, metric_g2_analytical, n_eigenvalues=10)

print("\n--- Grid size = 7 ---")
result_g2_7 = compute_spectrum(7, metric_g2_analytical, n_eigenvalues=10)

# Apply calibration
lambda1_g2_calibrated_5 = result_g2_5['lambda1'] * calibration_factor_5
lambda1_g2_calibrated_7 = result_g2_7['lambda1'] * calibration_factor_7

print(f"\n--- Calibrated Results ---")
print(f"  Grid 5: λ₁(K₇) = {lambda1_g2_calibrated_5:.6f}")
print(f"  Grid 7: λ₁(K₇) = {lambda1_g2_calibrated_7:.6f}")
print(f"  Theory: λ₁(K₇) = 1/g_ii = {1/G_DIAGONAL:.6f}")

In [ ]:
# @title 8. The Critical Product: λ₁ × H*

print("\n" + "═" * 60)
print("THE GIFT PREDICTION: λ₁ × H*")
print("═" * 60)

# Use best calibrated result (grid 7)
lambda1_K7 = lambda1_g2_calibrated_7
product = lambda1_K7 * H_STAR

print(f"\nλ₁(K₇) = {lambda1_K7:.6f}")
print(f"H* = {H_STAR}")
print(f"\n  λ₁ × H* = {product:.4f}")

# Compare to targets
dev_13 = abs(product - 13) / 13 * 100
dev_14 = abs(product - 14) / 14 * 100

print(f"\n--- Comparison to Targets ---")
print(f"  Target 13 (dim(G₂)-1): deviation = {dev_13:.2f}%")
print(f"  Target 14 (dim(G₂)):   deviation = {dev_14:.2f}%")

# Theoretical prediction for constant metric
product_theory = (1 / G_DIAGONAL) * H_STAR
print(f"\n--- Theoretical (Constant Metric) ---")
print(f"  λ₁ = 1/g_ii = {1/G_DIAGONAL:.6f}")
print(f"  λ₁ × H* = {product_theory:.4f}")

# This equals (65/32)^(-1/7) × 99
print(f"\n  = (65/32)^(-1/7) × 99")
print(f"  = (32/65)^(1/7) × 99")
print(f"  = {(32/65)**(1/7):.6f} × 99")
print(f"  = {(32/65)**(1/7) * 99:.4f}")

In [ ]:
# @title 9. Richardson Extrapolation for Better Accuracy

print("\n" + "═" * 60)
print("RICHARDSON EXTRAPOLATION")
print("═" * 60)
print("\nUsing grids 5, 7, 9 to extrapolate to h→0")

# Grid 9 (larger - may take a moment)
print("\n--- Grid size = 9 ---")
result_g2_9 = compute_spectrum(9, metric_g2_analytical, n_eigenvalues=10)
result_flat_9 = compute_spectrum(9, metric_flat, n_eigenvalues=10)
calibration_factor_9 = 1.0 / result_flat_9['lambda1']
lambda1_g2_calibrated_9 = result_g2_9['lambda1'] * calibration_factor_9

# Richardson extrapolation (assuming O(h²) error)
# λ_true ≈ λ_h + C·h² + O(h⁴)
# With two estimates at h1 and h2:
# λ_extrap = (h1² λ(h2) - h2² λ(h1)) / (h1² - h2²)

h5 = 2*np.pi / 5
h7 = 2*np.pi / 7
h9 = 2*np.pi / 9

# Extrapolate using grids 7 and 9
lambda1_extrap = (h7**2 * lambda1_g2_calibrated_9 - h9**2 * lambda1_g2_calibrated_7) / (h7**2 - h9**2)

print(f"\n--- Extrapolated Result ---")
print(f"  λ₁(grid 5)  = {lambda1_g2_calibrated_5:.6f}")
print(f"  λ₁(grid 7)  = {lambda1_g2_calibrated_7:.6f}")
print(f"  λ₁(grid 9)  = {lambda1_g2_calibrated_9:.6f}")
print(f"  λ₁(extrap)  = {lambda1_extrap:.6f}")
print(f"  λ₁(theory)  = {1/G_DIAGONAL:.6f}")

product_extrap = lambda1_extrap * H_STAR
print(f"\n  λ₁ × H* (extrapolated) = {product_extrap:.4f}")

In [ ]:
# @title 10. Analysis: Why λ₁ × H* ≠ 14?

print("\n" + "═" * 60)
print("ANALYSIS: THE CONSTANT METRIC RESULT")
print("═" * 60)

print("""
For the ANALYTICAL G₂ metric from gift-core:
  • g_ij = (65/32)^(1/7) × δ_ij  (CONSTANT diagonal)
  • This is a CONFORMALLY FLAT metric on T⁷
  
For a conformally flat metric g = Ω² g_flat:
  • Ω² = (65/32)^(1/7) ≈ 1.1115
  • λ_g = λ_flat / Ω² = 1 / 1.1115 ≈ 0.8996
  • λ_g × H* = 0.8996 × 99 ≈ 89.06

This is NOT 13 or 14!

═══════════════════════════════════════════════════════════════
THE KEY INSIGHT:
═══════════════════════════════════════════════════════════════

The analytical G₂ metric from gift-core is the STANDARD G₂ form
scaled to satisfy det(g) = 65/32. But:

1. A TRUE K₇ is NOT a flat torus - it has non-trivial topology
2. The metric must be POSITION-DEPENDENT to encode the K₇ geometry
3. The spectral gap λ₁ = 14/H* comes from the GLOBAL topology,
   not just the local metric

The constant metric approximation is too simple!
""")

print(f"\n--- Numerical Results ---")
print(f"Constant G₂ metric: λ₁ × H* = {product_theory:.2f}")
print(f"GIFT prediction:    λ₁ × H* = 14")
print(f"Ratio: {product_theory / 14:.4f}")

In [ ]:
# @title 11. What Would Give λ₁ × H* = 14?

print("\n" + "═" * 60)
print("REVERSE ENGINEERING: WHAT METRIC GIVES λ₁ × H* = 14?")
print("═" * 60)

# If λ₁ × H* = 14, then λ₁ = 14/99
target_lambda1 = 14 / H_STAR
print(f"\nTarget: λ₁ = 14/{H_STAR} = {target_lambda1:.6f}")

# For constant diagonal metric g_ii = c:
# λ₁ = 1/c (on T⁷ with period 2π)
# So we need c = 1/λ₁ = 99/14
required_g_ii = 1 / target_lambda1
print(f"\nFor constant diagonal metric:")
print(f"  Required g_ii = 1/λ₁ = {H_STAR}/14 = {required_g_ii:.6f}")

# What det(g) does this give?
required_det = required_g_ii ** 7
print(f"  This gives det(g) = g_ii^7 = {required_det:.2f}")
print(f"  GIFT requires det(g) = 65/32 = {DET_G_TARGET}")

print(f"\n  Ratio: {required_det / DET_G_TARGET:.2f}")
print(f"  The required det(g) is {required_det/DET_G_TARGET:.0f}× larger than GIFT!")

print("""
═══════════════════════════════════════════════════════════════
CONCLUSION:
═══════════════════════════════════════════════════════════════

A CONSTANT metric satisfying det(g) = 65/32 CANNOT give λ₁ × H* = 14.

The GIFT prediction λ₁ = 14/99 must come from:
  1. Non-trivial K₇ topology (not T⁷)
  2. Position-dependent metric encoding the TCS construction
  3. Or a different definition of λ₁ (e.g., on 1-forms, or Hodge)

NEXT STEPS:
  • Implement TCS metric with varying geometry along the neck
  • Try the actual Joyce construction with singularities
  • Compare Δ₀ (scalar) vs Δ₁ (1-forms) on proper geometry
""")

In [ ]:
# @title 12. Export Results

results = {
    'version': 'v3_analytical',
    'method': 'Laplace-Beltrami on 7D grid with constant G2 metric',
    'metric': {
        'type': 'constant_diagonal',
        'g_ii': G_DIAGONAL,
        'det_g': DET_G_TARGET,
    },
    'calibration': {
        'reference': 'T7_flat',
        'lambda1_theory': 1.0,
    },
    'results': {
        'grid_5': {
            'lambda1_raw': result_g2_5['lambda1'],
            'lambda1_calibrated': lambda1_g2_calibrated_5,
            'product': lambda1_g2_calibrated_5 * H_STAR,
        },
        'grid_7': {
            'lambda1_raw': result_g2_7['lambda1'],
            'lambda1_calibrated': lambda1_g2_calibrated_7,
            'product': lambda1_g2_calibrated_7 * H_STAR,
        },
        'grid_9': {
            'lambda1_raw': result_g2_9['lambda1'],
            'lambda1_calibrated': lambda1_g2_calibrated_9,
            'product': lambda1_g2_calibrated_9 * H_STAR,
        },
        'extrapolated': {
            'lambda1': lambda1_extrap,
            'product': product_extrap,
        },
        'theoretical': {
            'lambda1': 1/G_DIAGONAL,
            'product': product_theory,
        }
    },
    'GIFT_constants': {
        'H_star': H_STAR,
        'dim_G2': DIM_G2,
        'target_product': 14,
    },
    'conclusion': 'Constant G2 metric gives lambda1 * H* ≈ 89, not 14. Need non-trivial K7 topology.'
}

with open('K7_spectral_v3_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print("Results saved to K7_spectral_v3_results.json")
print("\n" + "═" * 60)
print("EXPERIMENT COMPLETE")
print("═" * 60)